# Коллаборативная фильтрация

ПАКЕТ SURPRISE
- используйте данные MovieLens 1M
- можно использовать любые модели из пакета
- получите RMSE на тестовом сете 0.87 и ниже

In [1]:
pip install surprise

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from surprise import Dataset
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise import BaselineOnly
from surprise import accuracy

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
ratings = ratings[:1000000]

In [3]:
ratings.shape

(1000000, 4)

In [4]:
movies_with_ratings = movies.join(ratings.set_index('movieId'), on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,4.0,4.0,1.113766e+09
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,10.0,5.0,9.488858e+08
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,14.0,4.5,1.442169e+09
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15.0,4.0,1.370810e+09
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,22.0,4.0,1.237623e+09


In [5]:
movies_with_ratings.shape

(1000000, 6)

In [6]:
# оставим только пользоватей, оценки и названия фильмов
data = pd.DataFrame(
    {'userId': movies_with_ratings.userId,
    'title': movies_with_ratings.title,
    'rating': movies_with_ratings.rating}
)

In [7]:
data.head()

,userId,title,rating
0,4.0,Toy Story (1995),4.0
1,10.0,Toy Story (1995),5.0
2,14.0,Toy Story (1995),4.5
3,15.0,Toy Story (1995),4.0
4,22.0,Toy Story (1995),4.0


минимум и максимум для преобразования дф pandas в дф surprise

In [8]:
data.rating.min() 

0.5

In [9]:
data.rating.max()

5.0

In [10]:
reader = Reader(rating_scale=(0.5,5.0)) 
df = Dataset.load_from_df(data, reader)

In [11]:
trainset, testset = train_test_split(df, test_size=0.3) #делим данные

In [12]:
# BaselineOnly - Алгоритм прогнозирования базовой оценки для данного пользователя и элемента.
model = BaselineOnly(bsl_options={'method':'als', 'reg_i':15, 'reg_u':20, 'n_epochs':25}, verbose=True) #'reg_i': Параметр регуляризации для элементов.'reg_u': Параметр регуляризации для пользователей. 
#'n_epochs':Номер итерации процедуры ALS.verbose ( bool ) - выводить ли сообщения трассировки оценки смещения, сходства
model.fit(trainset)

Estimating biases using als...


In [13]:
test_pred = model.test(testset)

In [14]:
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.8864


0.8863575620136597

In [16]:
# Знаменитый алгоритм SVD , популяризированный Саймоном Функом во время приза Netflix Prize. Когда базовые показатели не используются, это эквивалентно вероятностной матричной факторизации
from surprise import SVD
model2 = SVD(n_factors=90, n_epochs=15)
model2.fit(trainset)

In [17]:
test_pred2 = model2.test(testset)
accuracy.rmse(test_pred2, verbose=True)

RMSE: 0.8663


0.8663103347763051